# DPO (Direct Preference Optimization)

### 0. 환경 설정

In [ ]:
!python -m pip install --upgrade pip
!pip install typing_extensions==4.7.1 --upgrade
!pip install transformers peft datasets bitsandbytes accelerate

In [ ]:
from huggingface_hub import login
import os
import torch

login(token="hf_xxx")

os.environ["WANDB_DISABLED"] = "true"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 1. 모델 로드

In [ ]:
model_name = "Bllossom/llama-3.2-Korean-Bllossom-3B"

In [ ]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

### 2. 학습 준비

##### (1) 모델 준비

In [ ]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "down_proj", "up_proj"],
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
from peft import prepare_model_for_kbit_training, get_peft_model

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

model.print_trainable_parameters()
model.train()
model.gradient_checkpointing_enable()

##### (2) 데이터 준비

In [ ]:
from datasets import load_dataset

dataset = load_dataset('mncai/orca_dpo_pairs_ko')

In [ ]:
# 데이터 전처리 함수 정의
def preprocess_text(sample):
    input_enc = tokenizer(sample["question"], padding="max_length", max_length=256, truncation=True)
    preferred_enc = tokenizer(sample["chosen"], padding="max_length", max_length=256, truncation=True)
    despreferred_enc = tokenizer(sample["rejected"], padding="max_length", max_length=256, truncation=True)

    return {
        "input_ids": input_enc["input_ids"],
        "attention_mask": input_enc["attention_mask"],
        "preferred_ids": preferred_enc["input_ids"],
        "despreferred_ids": despreferred_enc["input_ids"]
    }

In [ ]:
tokenized_dataset = dataset["train"].map(
    preprocess_text,
    remove_columns=["id", "system", "question", "chosen", "rejected"]
)

tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "preferred_ids", "despreferred_ids"])

In [ ]:
def collate_fn(batch):
    input_ids = torch.stack([item["input_ids"].clone().detach() for item in batch])
    attention_mask = torch.stack([item["attention_mask"].clone().detach() for item in batch])
    
    max_length = max(max(len(item['preferred_ids']) for item in batch), 1)

    preferred_ids = torch.stack([
        torch.tensor(
            item["preferred_ids"].tolist() + [tokenizer.pad_token_id] * (max_length - len(item['preferred_ids'])),
            dtype=torch.long
        ) if isinstance(item["preferred_ids"], torch.Tensor) else
        torch.tensor(
            item["preferred_ids"] + [tokenizer.pad_token_id] * (max_length - len(item["preferred_ids"])),
            dtype=torch.long
        )
        for item in batch
    ]).clone().detach()

    despreferred_ids = torch.stack([
        torch.tensor(
            item["despreferred_ids"].tolist() + [tokenizer.pad_token_id] * (max_length - len(item['despreferred_ids'])),
            dtype=torch.long
        ) if isinstance(item["despreferred_ids"], torch.Tensor) else
        torch.tensor(
            item["despreferred_ids"] + [tokenizer.pad_token_id] * (max_length - len(item["despreferred_ids"])),
            dtype=torch.long
        )
        for item in batch
    ]).clone().detach()

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "preferred_ids": preferred_ids,
        "despreferred_ids": despreferred_ids
    }

##### (3) Trainer 준비 및 train()

In [ ]:
from transformers import Trainer
import torch.nn.functional as F

class DTOTrainer(Trainer):
    def compute_loss(self, model, inputs, beta=0.1, *args, **kwargs):
        input_ids = inputs["input_ids"].to(model.device)
        attention_mask = inputs["attention_mask"].to(model.device)
        preferred_ids = inputs["preferred_ids"].to(model.device)
        despreferred_ids = inputs["despreferred_ids"].to(model.device)

        preferred_outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=preferred_ids)
        despreferred_outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=despreferred_ids)
        
        preferred_loss = preferred_outputs.loss
        despreferred_loss = despreferred_outputs.loss

        loss = -F.logsigmoid(beta * (despreferred_loss - preferred_loss)).mean()
        return loss

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./dpo_llama3_korean",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=1e-4,
    num_train_epochs=3,
    save_total_limit=2,
    save_strategy="steps",
    save_steps=200,
    logging_steps=50,
    remove_unused_columns=False,
    fp16=True,
    optim="adamw_bnb_8bit",
    max_grad_norm=0    
)

In [ ]:
trainer = DTOTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=collate_fn
)

### 3. 학습

In [ ]:
trainer.train()

### 4. 학습된 모델 응답 생성

In [ ]:
from peft import PeftModel

checkpoint_path = "./dpo_llama3_korean/checkpoint-xxx"

model = PeftModel.from_pretrained(model, checkpoint_path)
model.eval()

sample_data = dataset["train"].select(range(5))

In [ ]:
def generate_response(question):
    inputs = tokenizer(question, return_tensors="pt", padding=True, truncation=True, max_length=256).to(model.device)
    
    with torch.no_grad():
        output_ids = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=256,
            temperature=0.7,
            top_p=0.9,
            do_sample=True
        )
        
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

In [ ]:
for i, example in enumerate(sample_data):
    question = example["question"]
    preferred_answer = example["chosen"]

    generated_reseponse = generate_response(question)

    print(f"{i}번째 질문: {question}")
    print(f"정답 (선호 응답): {preferred_answer}")
    print(f"실제 모델 응답: {generated_reseponse}")
    print("=" * 100)